### Import Libraries

In [54]:
import warnings
warnings.filterwarnings("ignore")

import yfinance as yf
import pandas as pd
import numpy as np
import talib as ta
from backtesting import Backtest, Strategy
import datetime

### Get data

In [55]:
tickers = ['MSFT', 'AAPL', 'NVDA', 'AMZN', 'GOOG', 'META', 'TSLA']

# Define the start and end dates
start_date = datetime.datetime(2013, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
data = {}
# Fetch historical data for each ticker
for ticker in tickers:
    data[ticker] = yf.download(ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [56]:
data['MSFT'].head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,27.250000,27.73,27.15,27.620001,22.532856,52899300
2013-01-03,27.629999,27.65,27.16,27.250000,22.231001,48294400
2013-01-04,27.270000,27.34,26.73,26.740000,21.814934,52521100


### Bollinger Bands Indicator

In [57]:
def calculate_bollinger_bands(close, window, stddev):
    upper_band, middle_band, lower_band = ta.BBANDS(close, timeperiod=window, nbdevup=stddev, nbdevdn=stddev)
    return upper_band, middle_band, lower_band

### Create Strategy Class using Backtesting.py 

In [58]:
class DoubleBollingerBand(Strategy):
    
    window = 20
    nbdev1 = 2
    nbdev2 = 1
    size = 1
    
    def init(self):
        """
        a1 > b1 > x > b2 > a2
        a1 : high upper band
        a2: low lower band
        x: MA
        b1: low upper band
        b2: high lower band
        """
        self.a1, self.x, self.a2 = self.I(calculate_bollinger_bands, self.data.Close,  self.window, self.nbdev1)
        self.b1, self.x, self.b2 = self.I(calculate_bollinger_bands, self.data.Close, self.window, self.nbdev2)
        
    def next(self):
        """
          Go Long if MA in A1-B1 Zone
          Go Short if MA in B2-A2 Zone
          Other: Close Position
        """
        
        if not self.position: # Current no position 
            
            if self.b1[-1] < self.data.Close[-1] < self.a1[-1]: # In Buy Zone
                self.buy(size=self.size)
            elif self.a2[-1] < self.data.Close[-1] < self.b2[-1]: # In Sell Zone
                self.sell(size=self.size)
            else:
                pass
        
        elif self.position.is_long: # Current long position 
            if self.a2[-1] < self.data.Close[-1] < self.b2[-1]: # In Sell Zone
                self.position.close()
                self.sell(size=self.size)
            elif not (self.b1[-1] < self.data.Close[-1] < self.a1[-1]): # Not In Buy Zone
                self.position.close()
            else:
                pass

        elif self.position.is_short: # Current short position 
            if self.b1[-1] < self.data.Close[-1] < self.a1[-1]: # In Buy Zone
                self.position.close()
                self.buy(size=self.size)
            elif not (self.a2[-1] < self.data.Close[-1] < self.b2[-1]): # Not In Sell Zone
                self.position.close()
            else:
                pass

### Backtesting

- Calculate: Total Return, Annual Return, Annual Volatlity, Sharpe Ratio, Sortino Ratio, Maximum Drawdown

In [59]:
# Parameters for backtesting
WINDOW = 20
NBDEV1 = 2
NBDEV2 = 1
SIZE = 1
INITIAL_CASH = 10000

In [60]:
strategy = DoubleBollingerBand
strategy.window = WINDOW
strategy.nbdev1 = NBDEV1
strategy.nbdev2 = NBDEV2
strategy.size = SIZE

In [61]:
# Run backtesting and save performance metrics
backtesting_results = {}

for symbol in data.keys():
    bt = Backtest(data[symbol], strategy, cash=INITIAL_CASH)
    output = bt.run()
    # bt.plot(filename=f'{symbol}.html')
    metrics = [output['Return [%]'], output['Return (Ann.) [%]'], output['Volatility (Ann.) [%]'], 
              output['Sharpe Ratio'], output['Sortino Ratio'], output['Max. Drawdown [%]']]
    backtesting_results[symbol] = metrics

In [62]:
# Create Pandas DataFrame to compare performance
backtesting_result_df = pd.DataFrame(backtesting_results)
backtesting_result_df.index = ['Total Return', 'Annual Return', 'Annual Volatlity', 'Sharpe Ratio', 'Sortino Ratio', 'Maximum Drawdown']
backtesting_result_df

,MSFT,AAPL,NVDA,AMZN,GOOG,META,TSLA
Total Return,-0.812901,0.641196,0.577375,-0.144520,-0.270949,-1.526202,2.127947
Annual Return,-0.074282,0.058205,0.052427,-0.013166,-0.024698,-0.139919,0.191881
Annual Volatlity,0.310505,0.181678,0.522185,0.217154,0.145689,0.444191,0.537772
Sharpe Ratio,0.000000,0.320376,0.100399,0.000000,0.000000,0.000000,0.356807
Sortino Ratio,0.000000,0.460633,0.158502,0.000000,0.000000,0.000000,0.520514
Maximum Drawdown,-1.369877,-0.442537,-1.451460,-0.609353,-0.574028,-2.010820,-1.356660


### End